In [1]:
%load_ext autoreload
%autoreload 2
%xmode Plain

Exception reporting mode: Plain


In [2]:
import pandas as pd
import plotly
import plotly.express as px 
import dtale
import numpy as np
import random
from bazaraki import utils
from tqdm import tqdm
import swifter
from pathlib import Path
import datacompy
from datetime import datetime, date
from parse import parse
from glob import glob 


/tmp/ipykernel_1707211/3232598961.py:11: UserWarning:

SparkPandasCompare currently only supports Numpy < 2.Please note that the SparkPandasCompare functionality will not work and currently is not supported.



In [3]:
pd.set_option('display.max_rows', 50)  # Disable row limit
pd.set_option('display.max_columns', 40)  # Disable column limit
pd.set_option('display.width', 20)  # Disable line width limit
pd.set_option('display.max_colwidth', 100)  # Disable column width limit
# Set the display precision to 2 decimal places  
pd.set_option('display.precision', 2)  
  
# Or set the float format to always show 2 decimal places  
pd.set_option('display.float_format', '{:.2f}'.format)  


In [4]:
pd.options.plotting.backend = "plotly"
plotly.io.renderers.default = "notebook_connected"


In [5]:
# group by city, count items for each city, present as dict


In [6]:
results = []
for file_name in sorted(glob("output/*.parquet")):
    print(f"Reading {file_name}")
    result = parse("output/{date} {time} {postfix}", file_name)
    if result is None:
        print(f"Failed to parse {file_name}")
        continue
    if result["postfix"].startswith("fast"):
        print(f"Skipping fast run")
        continue
    df = utils.read_df(file_name)
    dt = date.fromisoformat(result["date"])
    df[["city", "district"]] = df.location.str.split(",", expand=True)
    rent_df = df.query("cat1 == 'Apartments, flats to rent'")
    sale_df = df.query("cat1 == 'Apartments, flats for sale'")
    item = dict(dt=dt)
    item.update({f"rent_count_{k}": v for k, v in rent_df.groupby('city').size().to_dict().items()})
    item.update({f"sale_count_{k}": v for k, v in sale_df.groupby('city').size().to_dict().items()})
    item.update({f"rent_median_price_{k}": v for k, v in rent_df.groupby('city').price.median().to_dict().items()})
    item.update({f"sale_median_price_{k}": v for k, v in sale_df.groupby('city').price.median().to_dict().items()})
    
    results.append(item)



Reading output/2024-12-12 18:34:25 real-estate-to-rent_real-estate-for-sale.parquet


Reading output/2024-12-14 11:44:22 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-15 18:00:14 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-16 23:13:52 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-17 21:31:02 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-18 23:01:10 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-19 22:09:26 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-22 21:49:27 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-23 19:08:45 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-27 17:28:17 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-29 13:00:58 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2025-01-02 12:05:37 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2025-01-09 16:40:56 real-estate-to-rent_real-estate-for-sale.

In [7]:
# create df from results
results_df = pd.DataFrame(results).set_index("dt")
results_df

,rent_count_Famagusta,rent_count_Larnaca,rent_count_Limassol,rent_count_Nicosia,rent_count_Paphos,sale_count_Famagusta,sale_count_Larnaca,sale_count_Limassol,sale_count_Nicosia,sale_count_Paphos,rent_median_price_Famagusta,rent_median_price_Larnaca,rent_median_price_Limassol,rent_median_price_Nicosia,rent_median_price_Paphos,sale_median_price_Famagusta,sale_median_price_Larnaca,sale_median_price_Limassol,sale_median_price_Nicosia,sale_median_price_Paphos
dt,,,,,,,,,,,,,,,,,,,,
2024-12-12,62,665,2115,1331,391,450,2413,6792,2258,979,650.00,1000.00,1900.00,825.00,1050.00,200000.00,230000.00,435000.00,215000.00,300000.00
2024-12-14,65,654,2104,1324,371,453,2437,6787,2238,982,650.00,1000.00,1900.00,850.00,1050.00,200000.00,230000.00,435000.00,215000.00,300000.00
2024-12-15,63,654,2077,1317,371,455,2424,6737,2221,977,650.00,1000.00,1900.00,850.00,1050.00,200000.00,230000.00,435000.00,215000.00,300000.00
2024-12-16,60,650,2053,1326,387,455,2439,6775,2244,978,650.00,1000.00,1900.00,850.00,1050.00,200000.00,230000.00,435000.00,215000.00,300000.00
2024-12-17,60,650,2071,1346,388,452,2444,6820,2293,975,650.00,1000.00,1900.00,850.00,1095.50,200000.00,230000.00,435000.00,210000.00,300000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-01,60,777,2376,1734,319,474,3076,7095,2532,1278,800.00,1150.00,2000.00,850.00,1150.00,210000.00,235000.00,435000.00,215000.00,299000.00
2025-10-04,62,786,2375,1731,320,470,3133,7135,2472,1216,795.00,1150.00,2000.00,850.00,1200.00,210000.00,230000.00,440000.00,214500.00,300000.00
2025-10-07,67,808,2351,1696,322,464,3268,7110,2535,1246,800.00,1150.00,2100.00,890.00,1150.00,209000.00,230000.00,440000.00,215000.00,300000.00


In [8]:
sale_cols = [ col for col in results_df.columns if "sale_count" in col]
results_df[sale_cols].plot()

In [9]:
cols = [ col for col in results_df.columns if "rent_count" in col]
results_df[cols].plot()

In [10]:
cols = [ col for col in results_df.columns if "rent_median_price" in col]
results_df[cols].plot()

In [11]:
cols = [ col for col in results_df.columns if "sale_median_price" in col]
results_df[cols].plot()

In [12]:
df.iloc[0].T

url                                                      https://www.bazaraki.com/adv/5941936_3-bedroom-apartment-to-rent/
title                                                                                          3-bedroom apartment to rent
price                                                                                                                 1200
original_price                                                                                                         NaN
price_per_sqm                                                                                                          NaN
location                                                                                         Paphos, Agios Dimitrianos
posted                                                                                                    10.10.2025 16:51
ad_id                                                                                                              5941936
reference_number